``https://github.com/hhk998402/Time-Series-Forecasting-SARIMAX/blob/master/Hemant_Sangam.ipynb``

<b>Approach used:</b> SARIMAX (Seasonal Autoregressive Integrated Moving Average with eXogeneous variables)

<b> Reason:</b> The data provided is seasonal, and it is a time series data with multiple exogeneous variables influencing the result. Hence, the optimal statistical model that can be applied to this task is SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tqdm import tqdm
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import itertools
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

Open and store the dataset as a pandas dataframe

In [ ]:
# get data
df = pd.read_csv("../complete-merged-df.csv", index_col=0, parse_dates=True)
df.head()

Prepare the data:

- Split the data into train and test samples
- Create `endog` and `exog` variables


In [ ]:
# split data into train and test
top_predictors = ['close','open', 'high', 'low', 'n-transactions', 'cost-per-transaction',
                  'Gold price', 'output-volume',  'USD-CNY Price', 'SVI', 'Wikiviews']
df = df[top_predictors]

train, test = train_test_split(df, test_size=0.2, shuffle=False)
# len(train), len(test)

# Variables
exog_data = train.drop(['close'], axis=1)

exog = sm.add_constant(exog_data)
endog = train[['close']]


Find the best order for the ARIMA model

In [ ]:
# Initial approximation of parameters
Qs = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
ps = range(0, 3)
D = 1
d = 1
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

# Model Selection
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
for param in parameters_list:
    try:
        model = SARIMAX(endog, exog=exog, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], 4)).fit(disp=-1)
    except ValueError:
        print('bad parameter combination:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])


In [ ]:
# Best Models
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by='aic', ascending=True).head())
print(best_model.summary())


Train the model with the train variables with order `(p=1, d=0, q=1)`

In [ ]:
# train model
mod = SARIMAX(endog, exog=exog, order=(1, 1, 2), seasonal_order=(0, 1, 1, 4))
fit_res = mod.fit(disp=False)
fit_res.summary()


### Breakdown of the summary order=(1, 0, 1)
- The Ljung-Box (L1) (Q) at lag 1 is 0, and the Prob(Q) is 0.99. Since the probability is above 0.05, we can't reject the null that the errors are white noise.
- Heteroscedasticity tests that the error residuals are homoscedastic or have the same variance. Our summary statistics show a test statistic of 0.65 and a p-value of 0.00, which means we reject the null hypothesis and our residuals show variance.
- Jarque-Bera tests for the normality of errors. It tests the null that the data is normally distributed against an alternative of another distribution. We see a test statistic of `13286.64` with a probability of 0, which means we reject the null hypothesis, and the data is not normally distributed. Also, as part of the Jarque-Bera test, we see the distribution has a slight negative skew and a large kurtosis.

In [ ]:
# adfuller test
print("Dickey–Fuller test:: p=%f" % sm.tsa.adfuller(fit_res.resid[13:])[1])


Display the residuals from the model fit in acf and pacf plots

In [ ]:
res = fit_res.resid
fig, ax = plt.subplots(2, 1, figsize=(15, 8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [ ]:
res_df = pd.DataFrame(fit_res.resid, columns=['resid'])
res_df.sort_values(by='resid', ascending=False).head(5)

The outlier in date `2017-12-22`?

In [ ]:
fit_res.plot_diagnostics(figsize=(12, 6))

### Residual analysis of the selected model
- We can see that the residuals have no trend and a fairly constant variance over time, just like white noise. 
- On the top right plot, the distribution of residuals is very close to a normal distribution. 
- This is further supported by the Q-Q plot on the bottom left that shows a fairly straight line that lies on y = x. 
- Finally, the correlogram shows no significant coefficients after lag 0, just like white noise. 

Therefore, from a graphical analysis, the residuals of this model resemble white noise.

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0, 1]   # corr
    mins = np.amin(np.hstack([forecast[:, None],
                              actual[:, None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:, None],
                              actual[:, None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    
    return({'mape': mape, 'me': me, 'mae': mae,
            'mpe': mpe, 'rmse': rmse, 
            'corr': corr, 'minmax': minmax})

## Now lets test the model's prediction

In [ ]:
# test model
first_predict, last_predict = test.iloc[0].name, test.iloc[-1].name

exog1 = (sm.add_constant(test).loc[first_predict:])
exog1 = exog1.drop(['close'], axis=1)

forecast = fit_res.forecast(steps=len(test), exog=exog1)
print(len(forecast), len(test))

result_data = pd.DataFrame(index=test.index, columns=['actual', 'pred'])
# result_data.head()

chk = 0
for i in tqdm(forecast):
    result_data.iloc[chk]["actual"] = df.iloc[df.index == test.iloc[chk].name]['close'].values[0]
    result_data.iloc[chk]["pred"] = i
    chk += 1
    
result_data.head(10)

In [ ]:
# compare the values at a random date from the dataset
sample_date = test.iloc[5].name
y_hat = df.loc[sample_date]['close']
y_pred = result_data[result_data.index == sample_date]["pred"].values[0]

# compare that date with the predicted date
print(F"{sample_date}\n\tActual {y_hat}\n\tPredicted {y_pred}")

# display the forecast accuracy metrics for the test set
forecast_accuracy(forecast, test['close'].values)

We can see the predicted value is quite close to the actual value on that date.

In [ ]:
# plot the test data and the predicted data
plt.plot(df['close'], label='test', color='blue')
plt.plot(result_data['pred'], label='predicted', color='red')

The blue label depicts the actual values, while the red label shows the prediction made. <br>
The plot shows us the model has predicted the values closely to the actual values.

### Forecast Tests
- Make a prediction using the model for the next `n-days/months/years`.
- The `get_new_data()` function gets the BPI from Yahoo Finance for the latest/given dates to compare with the predictions.
- Create a `future` DataFrame with the actual and forecasted values
- Create a mask for a short time frame and visualize it in a plot
- Calculate metrics

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

n_years = 2
n_days = 365*n_years+1

exog_last = sm.add_constant(df.drop(['close'], axis=1))[-n_days:]
start_index = exog.index.max().date() + relativedelta(days=1)
end_index = date(start_index.year+n_years, start_index.month, start_index.day)
pred = fit_res.predict(start=start_index, end=end_index, exog=exog_last)

print(start_index, end_index)
try:
    print(f"Forecast {pred.iloc[pred.index == '2022-01-05']}")
    # print(f"Actual {df.loc['2021-01-05']['close']}")
except IndexError as e:
    print(f"Cannot find the forecast date (max {end_index})")
    pass

In [ ]:
import yfinance as yf
from yahoofinancials import YahooFinancials

# get new data from yahoo finance
def get_new_data(ticker, start_date, end_date=date.today()):
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return data

In [ ]:
yf_df = get_new_data('BTC-USD', '2020-12-31', '2022-03-20')

# create a new dataframe with the actual and predicted values
future = pd.DataFrame(columns=['actual', 'pred', 'train', 'forecast'])
future['forecast'] = pred
future["actual"] = yf_df['Close']
future["pred"] = result_data['pred']
future['train'] = df['close']
future
# yf_df['Close']

In [ ]:
# compare predicted values with actual values
mask = (future.index >= '2021-12-01') & (future.index <= '2022-04-30')
filter = future.loc[mask]
filter.plot(figsize=(12, 6), title='Bitcoin Price Prediction', grid=True, legend=True, fontsize=12)

# The lower the MSE, the better the forecast.
forecast_accuracy(filter['forecast'], filter['actual'])

In [ ]:
future.plot(figsize=(12, 6))

# yf_df.iloc[yf_df.index >= '2021-12-01']['Close'].plot(figsize=(12, 6), label='actual', color='blue')
# result_data.iloc[result_data.index >= '2021-12-01']['pred'].plot(figsize=(12, 6), label='predicted', color='red')
# pred.iloc[pred.index >=
#           '2021-12-01'].plot(figsize=(12, 6), label='Forecasted', color='green')


The results indicate that the model is still a little rough and not something we should use as trading advice, but that was not unexpected due to the extremely volatile nature of cryptocurrencies, especially in the last 6 months.

It is probably also not such a good idea to try and predict 6 months into the future as we can see how insane even the 80% confidence interval becomes out this far. Maybe sticking to 1 month advance predictitons is more sensible. Or maybe even predicting on a daily basis.